In [46]:
import django_initializer
import petl as etl
from django.conf import settings


DATA_DIR = settings.BASE_DIR / ".." / "data"
DATA_DIR


RuntimeError: populate() isn't reentrant

In [ ]:
from decimal import Decimal

null_values = {
    "address not listed": None,
    "not listed": None,
    "street not listed": None,  # could mean street is gone or destroyed or didn't exist yet.
}

constraints = [dict(name="street_name", field="street", test=str, optional=False)]
residents = (
    etl.fromcsv(DATA_DIR / "residents.csv")
    # .convert("no", Decimal)
    .convert("1920", null_values)
    .convert("1921", null_values)
    .convert("1922", null_values)
    .convert("1923", null_values)
    .convert("1920 census", null_values)
)
residents


street,no,1920,1921,1922,1923,1920 census,losses,notes
Admiral Blvd.,3,None,None,None,R.W. Williams,,,
Admiral Blvd.,800,None,"Pardillo, Victor",None,None,"Padilla, Victor (25) laborer",,
Admiral Blvd.,801,None,Hooper Bros.*,None,None,,,
Admiral Blvd.,807,None,None,"Hoper Bros, whol. Tea",None,,,
Admiral Blvd.,807,Tulsa Coffee Co.*,Tulsa Coffee Co.*,Tulsa Coffee Co.,None,,,


In [ ]:
problems = etl.validate(residents, constraints=constraints)
problems.lookall()


+------+-----+-------+-------+-------+
| name | row | field | value | error |
+======+=====+=======+=======+=======+

In [ ]:
from addresses.models import Address, Street
from petl import dicts

year_columns = (
    "1920",
    "1921",
    "1922",
    "1923",
    "1920 census",
)

for row in dicts(residents):
    street, street_created = Street.get_or_create_from_name(row["street"])

    building_number = row["no"]
    if building_number == "" or street is None:
        # logger.info("no building number, skipped: %s", row)
        continue

    address, address_created = Address.get_or_create_from_unique(
        street=street, number=building_number
    )

    print(address, address_created)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/app/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2539/2884868851.py", line 20, in <module>
    address, address_created = Address.get_or_create_from_unique(
  File "/app/greenwood_history/apps/addresses/models.py", line 143, in get_or_create_from_unique
  File "/app/.venv/lib/python3.10/site-packages/django/db/models/manager.py", line 87, in manager_method
    return getattr(self.get_queryset(), name)(*args, **kwargs)
  File "/app/.venv/lib/python3.10/site-packages/django/db/models/query.py", line 916, in get_or_create
    return self.get(**kwargs), False
  File "/app/.venv/lib/python3.10/site-packages/django/db/models/query.py", line 623, in get
    clone = self._chain() if self.query.combinator else self.filter(*args, **kwargs)
  File "/app/.venv/lib/python3.10/site-packages/django/db/models/query.py", line 1436, in f